In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Models
from lightgbm import LGBMRegressor

# Metrics
from sklearn.metrics import mean_squared_error

/kaggle/input/testing-data/test.csv
/kaggle/input/submit-prediction/sample_submission.csv
/kaggle/input/training-data/train.csv


In [2]:
test=pd.read_csv("/kaggle/input/testing-data/test.csv")
train=pd.read_csv("/kaggle/input/training-data/train.csv")
submit_prediction=pd.read_csv("/kaggle/input/submit-prediction/sample_submission.csv")

In [3]:
train.head()


,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


In [4]:
train.info

<bound method DataFrame.info of             id  age  gender   course  study_hours  class_attendance  \
0            0   21  female     b.sc         7.91              98.8   
1            1   18   other  diploma         4.95              94.8   
2            2   20  female     b.sc         4.68              92.6   
3            3   19    male     b.sc         2.00              49.5   
4            4   23    male      bca         7.65              86.9   
...        ...  ...     ...      ...          ...               ...   
629995  629995   18  female   b.tech         4.86              70.7   
629996  629996   21  female       ba         7.08              54.4   
629997  629997   24    male      bca         0.64              44.2   
629998  629998   20    male    b.com         1.54              75.1   
629999  629999   18   other   b.tech         3.94              75.3   

       internet_access  sleep_hours sleep_quality   study_method  \
0                   no          4.9       avera

In [5]:
TARGET="exam_score"
X=train.drop(columns=[TARGET])
Y=train[TARGET]

In [6]:
X=X.drop(columns=["id"],errors="ignore")
test_X=test.drop(columns=["id"],errors="ignore")

In [7]:
cat_cols = X.select_dtypes(include="object").columns
print("Categorical columns:", list(cat_cols))

Categorical columns: ['gender', 'course', 'internet_access', 'sleep_quality', 'study_method', 'facility_rating', 'exam_difficulty']


In [8]:
cat_cols = X.select_dtypes(include="object").columns

for col in cat_cols:
    X[col] = X[col].astype("category")
    test_X[col] = test_X[col].astype("category")
    test_X[col] = test_X[col].cat.set_categories(X[col].cat.categories)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(
    X, Y,
    test_size=0.2,
    random_state=42
)

In [13]:
import lightgbm as lgb
from lightgbm import LGBMRegressor

model = LGBMRegressor(
    n_estimators=5000,
    learning_rate=0.03,
    num_leaves=63,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    
    callbacks=[
        lgb.early_stopping(stopping_rounds=200),
        lgb.log_evaluation(period=100)
    ]
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011850 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 602
[LightGBM] [Info] Number of data points in the train set: 504000, number of used features: 11
[LightGBM] [Info] Start training from score 62.482335
Training until validation scores don't improve for 200 rounds
[100]	valid_0's l2: 80.5478
[200]	valid_0's l2: 77.2861
[300]	valid_0's l2: 76.9967
[400]	valid_0's l2: 76.8741
[500]	valid_0's l2: 76.7691
[600]	valid_0's l2: 76.7027
[700]	valid_0's l2: 76.6457
[800]	valid_0's l2: 76.6134
[900]	valid_0's l2: 76.5854
[1000]	valid_0's l2: 76.5663
[1100]	valid_0's l2: 76.5462
[1200]	valid_0's l2: 76.5288
[1300]	valid_0's l2: 76.5231
[1400]	valid_0's l2: 76.5147
[1500]	valid_0's l2: 76.4982
[1600]	valid_0's l2: 76.4832
[1700]	valid_0's l2: 76.4798
[1800]	valid_0's l2: 76.4764
[1900]	valid_0's l

LGBMRegressor(colsample_bytree=0.8, learning_rate=0.03, n_estimators=5000,
              num_leaves=63, random_state=42, subsample=0.8)

In [14]:
from sklearn.metrics import mean_squared_error

val_preds = model.predict(X_val)
mse = mean_squared_error(y_val, val_preds)
rmse = np.sqrt(mse)

print("Validation RMSE:", rmse)

Validation RMSE: 8.74482790277721


In [19]:
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

test_preds = np.zeros(len(test_X))

for fold, (tr_idx, val_idx) in enumerate(kf.split(X)):
    print(f"Training fold {fold + 1}")

    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = Y.iloc[tr_idx], Y.iloc[val_idx]

    model = LGBMRegressor(
        n_estimators=5000,
        learning_rate=0.03,
        num_leaves=63,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=[
            lgb.early_stopping(stopping_rounds=200),
            lgb.log_evaluation(period=100)
        ]
    )

    test_preds += model.predict(test_X) / kf.n_splits

Training fold 1
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011624 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 603
[LightGBM] [Info] Number of data points in the train set: 504000, number of used features: 11
[LightGBM] [Info] Start training from score 62.482335
Training until validation scores don't improve for 200 rounds
[100]	valid_0's l2: 80.5303
[200]	valid_0's l2: 77.2857
[300]	valid_0's l2: 76.9757
[400]	valid_0's l2: 76.849
[500]	valid_0's l2: 76.7744
[600]	valid_0's l2: 76.7076
[700]	valid_0's l2: 76.6497
[800]	valid_0's l2: 76.6034
[900]	valid_0's l2: 76.5627
[1000]	valid_0's l2: 76.5419
[1100]	valid_0's l2: 76.526
[1200]	valid_0's l2: 76.5156
[1300]	valid_0's l2: 76.5018
[1400]	valid_0's l2: 76.4927
[1500]	valid_0's l2: 76.4903
[1600]	valid_0's l2: 76.4892
[1700]	valid_0's l2: 76.4848
[1800]	valid_0's l2: 76.4824
[190

In [20]:
test_preds = model.predict(test_X)
print(test_preds)

[71.28321954 68.8952529  87.67024215 ... 90.90798028 54.42116217
 67.55447211]


In [21]:
print(submit_prediction.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270000 entries, 0 to 269999
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   id          270000 non-null  int64
 1   exam_score  270000 non-null  int64
dtypes: int64(2)
memory usage: 4.1 MB
None


In [22]:
submit_prediction["exam_score"].describe()

count    270000.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: exam_score, dtype: float64

In [23]:
Y.describe()

count    630000.000000
mean         62.506672
std          18.916884
min          19.599000
25%          48.800000
50%          62.600000
75%          76.300000
max         100.000000
Name: exam_score, dtype: float64

In [25]:
submit_prediction = pd.DataFrame({
    "id": test["id"],
    "exam_score": test_preds
})

submit_prediction.to_csv("submission2.csv", index=False)

In [26]:
submit_prediction.head()

,id,exam_score
0,630000,71.283220
1,630001,68.895253
2,630002,87.670242
3,630003,55.295789
4,630004,47.272543


In [27]:
submit_prediction.tail()

,id,exam_score
269995,899995,59.464421
269996,899996,40.636635
269997,899997,90.907980
269998,899998,54.421162
269999,899999,67.554472


In [28]:
submit_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270000 entries, 0 to 269999
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          270000 non-null  int64  
 1   exam_score  270000 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 4.1 MB
